Analyse des erreurs du lemmatiseur
==============================

## Étapes

1. Parsage
2. Comparaison Synchronie vs Diachronie (Ablatif comme adverbe, Participe comme adjectif, etc.)
3. Décompte du nombre de lemmes nouveaux

## 1. Parsage

In [1]:
errors = [
    #(source, target, count)
]
preds = []
trues = []
with open("LemmatisationReport.md") as f:
    started = False
    last_true = None
    for line in f:
        if not line.strip():
            continue
        if line.endswith("lemma  Confusion Matrix\n"):
            started = True
            continue
        elif line.startswith("#"):
            started = False
            continue
        elif "Expected" in line:
            continue
        elif "|---" in line:
            continue
            
        if not started:
            continue
            
        # We are no dealing only with the table
        true, _, pred, cnt = [x.strip() for x in line.strip().split("|") if x]
        if not last_true:
            last_true = true
        preds.append(pred)
        if true:
            last_true = true
            trues.append(true)
        else:
            true = last_true
        
        if "界" in true and "界" in pred:
            true = true.split("界")[0]
            pred = pred.split("界")[0]
            
        errors.append((true, pred, int(cnt)))

## 2. Comparaison Synchronie vs. Diachronie

### 2.1 Chargement du lemmatiseur

In [2]:
from pycollatinus import Lemmatiseur
lemmatizer = Lemmatiseur()

### 2.2 Lemmatisation de tous les lemmes

In [3]:
from collections import defaultdict
lemmatisations = {}
for lemma in set(preds + trues):
    lemmatisations[lemma] = list(lemmatizer.lemmatise(lemma))

### 2.3 Comparaison suivant certaines catégories

In [4]:
def is_particip_parf(lemma):
    for possibility in lemmatisations[lemma]:
        if possibility["morph"] == "nominatif masculin singulier participe parfait passif":
            return possibility["lemma"]
        elif possibility["morph"] == "accusatif masculin singulier participe parfait passif":
            return possibility["lemma"]
    return -1

def is_particip_pres(lemma):
    for possibility in lemmatisations[lemma]:
        if possibility["morph"] == "nominatif masculin singulier participe présent actif":
            return possibility["lemma"]
        #elif possibility["morph"] == "accusatif masculin singulier participe présent actif":
        #    return possibility["lemma"]
    return -3

def is_verb(lemma):
    for possibility in lemmatisations[lemma]:
        if possibility["morph"] == "1ère singulier indicatif présent actif":
            return possibility["lemma"]
    return -2

def share_nominatif(lemma):
    for possibility in lemmatisations[lemma]:
        if possibility["morph"] in (
            "nominatif singulier", "nominatif neutre singulier", "nominatif féminin singulier",
            "nominatif masculin singulier"
        ):
            return possibility["lemma"]
    return lemma

errors_repartition = {
    "part-parf-as-verb": 0,
    "verb-as-part-parf": 0,
    "part-pres-as-verb": 0,
    "verb-as-part-pres": 0,
    "nom-as-something-else": 0,
    "something-else-as-nom": 0,
    "passif-as-actif": 0,
    "actif-as-passif": 0,
    "same-nominatif": 0,
    "total": 0
}

for true, pred, cnt in errors:
    errors_repartition["total"] += cnt
    
    if true not in lemmatisations or pred not in lemmatisations:
        continue
    
    is_verb_pred = is_verb(pred)
    is_verb_true = is_verb(true)
    
    if pred in ("qui", "quod", "quis", "quam", "qua"):
        continue
    if true+"r" == pred:
        errors_repartition["actif-as-passif"] += cnt
    elif true == pred+"r":
        errors_repartition["passif-as-actif"] += cnt
    elif is_particip_parf(true) == is_verb_pred:
        #print(true, pred)
        errors_repartition["part-parf-as-verb"] += cnt
    elif is_verb_true == is_particip_parf(pred):
        #print(true, pred)
        errors_repartition["verb-as-part-parf"] += cnt
    elif is_particip_pres(true) == is_verb_pred:
        #print(true, pred)
        errors_repartition["part-pres-as-verb"] += cnt
    elif is_verb_true == is_particip_pres(pred):
        #print(true, pred)
        errors_repartition["verb-as-part-pres"] += cnt
    elif share_nominatif(true) == pred:
        errors_repartition["nom-as-something-else"] += cnt
    elif true == share_nominatif(pred):
        errors_repartition["something-else-as-nom"] += cnt
    elif share_nominatif(true) == share_nominatif(pred):
        errors_repartition["same-nominatif"] += cnt
    
print(errors_repartition)

{'part-parf-as-verb': 282, 'verb-as-part-parf': 174, 'part-pres-as-verb': 52, 'verb-as-part-pres': 44, 'nom-as-something-else': 86, 'something-else-as-nom': 136, 'passif-as-actif': 23, 'actif-as-passif': 28, 'same-nominatif': 76, 'total': 4207}


### 2.4 Affichage relatif

In [5]:
print(errors_repartition)

support = 169822
# 169822-0.9752*169822
total_diachronical = 0
for type_error, cnt_errors in errors_repartition.items():
    if type_error=="total":
        continue
    #print("{}: {:.4}".format(type_error, cnt_errors/support))
    print("{}: {:.2f} % of errors".format(type_error, cnt_errors*100/errors_repartition["total"]))
    total_diachronical += cnt_errors

print("---")
print("{}: {:.2f} % of errors".format(type_error, total_diachronical*100/errors_repartition["total"]))

{'part-parf-as-verb': 282, 'verb-as-part-parf': 174, 'part-pres-as-verb': 52, 'verb-as-part-pres': 44, 'nom-as-something-else': 86, 'something-else-as-nom': 136, 'passif-as-actif': 23, 'actif-as-passif': 28, 'same-nominatif': 76, 'total': 4207}
part-parf-as-verb: 6.70 % of errors
verb-as-part-parf: 4.14 % of errors
part-pres-as-verb: 1.24 % of errors
verb-as-part-pres: 1.05 % of errors
nom-as-something-else: 2.04 % of errors
something-else-as-nom: 3.23 % of errors
passif-as-actif: 0.55 % of errors
actif-as-passif: 0.67 % of errors
same-nominatif: 1.81 % of errors
---
total: 21.42 % of errors


## 3. Lemmes existants, lemmes inexistants

### 3.1 Chargement de la base de lemmes existants

In [6]:
lemmatas = []
with open("LASLA-Lemmas/out.txt") as f:
    for lineno, line in enumerate(f):
        if lineno == 0:
            continue
        line = line.strip().split("\t")[0]
        if line[-1].isnumeric():  # Remove disambiguation index
            line=line[:-1]
        lemmatas.append(line.lower())
        
lemmatas = set(lemmatas)
print(len(lemmatas))

25185


### 3.2 Calcul du nombre d'inconnus

In [8]:
unknown = 0
totally_unknown = 0
unknown_pred = []

for true, pred, cnt in errors:
    if pred not in lemmatas:
        unknown += cnt
        if pred not in lemmatisations:
            totally_unknown += cnt
            print(pred)
        else:
            print(pred)
        unknown_pred.append((true, pred))
        
print("{}: {:.2f} % of errors ({})".format("Unknown", unknown*100/errors_repartition["total"], unknown))
print("{} (included in Collatinus): {:.2f} % of errors ({})".format(
    "Unknown", totally_unknown*100/errors_repartition["total"], totally_unknown
))

bons
eumo
utertia
utentia
moenio
ardes
qui界que
quisquisque
si界uolo
bella
nocturne
nocturna
simplicius
simplicus
teucrus
nitus
reuersus
tonde
innoto
insta
promero
impertus
imperto
attinus
attinae
turicrema
myrrhena
caesea
caesio
frix
frigens
hercynia
solutus界sum
sanes
memacenes
memacenus
laris
uiges
clito
cliton
mytilenus
licita
misella
misellis
opificum
narniensis
summa界sumptus
uenumdo
subpar
supara
lano
essurio
rixus
inaris
inarimen
antisto
titia
apelle
facisso
priauo
captiue
lotus
typanus
saepeo
impauidi
orbor
seuiro
excludo界sum
aprilia
quatuordeo
alanis
utrinque
libyci
superincumbus
honoris
carius
lechaeus
hirquus
baii
atrienses
perdelis
accliue
rosetus
cibo
multifarius
aletrinati
internito
oblideo
singlariter
calcio
pulmus
nasut
sterquilinus
commoeto
borestorus
exento
hirrius
suptero
cum界que
philiteus
sallustus
semustus
pleades
sameramis
lai
pammeni
antecellus
carnificus
resico
rhinocer
maedus
melicher
acosmus
dorco
charito
mius
cataplexus
larinus
excutido
cappadocas
duplica
aeeta
